In [13]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [14]:
length=100
batch_size=64
learning_rate=0.000001
epocas=5

In [15]:
#plt.plot(dadosTreinoValidacao[5200:5500],'r')
#plt.plot(targets[5200:5500],'b')

In [16]:
dados_teste=np.load('dadosTeste.npy')
dadosTreinoValidacao=np.load('dadosTreinoValidacao.npy')

print(np.shape(dadosTreinoValidacao),
np.shape(dados_teste))

dadosTreinoValidacao[5200:5500]

(100000,) (8937,)


array([ 4.,  8.,  1.,  5.,  2.,  1.,  6., 10.,  7.,  4.,  3.,  0.,  4.,
        5.,  3.,  5.,  9.,  9.,  7.,  5.,  4.,  8.,  1.,  4.,  2.,  1.,
        5., 10.,  7.,  4.,  3.,  0.,  4.,  5.,  4.,  5.,  9.,  9.,  7.,
        5.,  4.,  7.,  1.,  4.,  2.,  1.,  5., 10.,  7.,  4.,  3.,  0.,
        4.,  5.,  4.,  5.,  9.,  9.,  7.,  5.,  5.,  7.,  1.,  4.,  2.,
        1.,  6., 10.,  6.,  4.,  3.,  0.,  4.,  5.,  4.,  5.,  9.,  9.,
        7.,  5.,  5.,  7.,  1.,  3.,  2.,  1.,  6., 11.,  6.,  4.,  3.,
        1.,  4.,  5.,  4.,  5.,  9.,  8.,  7.,  5.,  5.,  7.,  1.,  2.,
        2.,  2.,  7., 11.,  6.,  4.,  2.,  1.,  4.,  5.,  4.,  5.,  9.,
        8.,  7.,  5.,  4.,  7.,  1.,  3.,  3.,  2.,  7., 11.,  7.,  4.,
        2.,  2.,  4.,  5.,  4.,  5.,  9.,  7.,  7.,  5.,  4.,  7.,  1.,
        4.,  3.,  2.,  8., 12.,  7.,  4.,  2.,  2.,  4.,  5.,  4.,  5.,
        9.,  7.,  7.,  5.,  5.,  6.,  1.,  3.,  3.,  2.,  8., 12.,  7.,
        3.,  2.,  3.,  4.,  4.,  4.,  5., 10.,  7.,  8.,  6.,  6

In [17]:
percentagem_dados=80

x_train = dadosTreinoValidacao[:80000]

x_validation = dadosTreinoValidacao[80000:]

print(np.shape(x_train), np.shape(x_validation))

(80000,) (20000,)


### Dados treino + função que prepara os dados TimeSeriesGenerator

In [18]:
def preparar_targets(x_train, length,batch_size):
    d=5
    salto=3 
    n=d+salto #8
    targets=[]
    for i in range(len(x_train)):
        soma = np.sum(x_train[d:n])
        d= d+1
        n= n+1
        targets.append(soma)
    del targets[-5:]
    
    targets = np.expand_dims(targets, axis=1)
    
    new = x_train[5:]
    inputs = np.expand_dims(new, axis=1)

    time_series_data = tf.keras.preprocessing.sequence.TimeseriesGenerator(inputs, targets, length= length, batch_size=batch_size)
    
    return time_series_data


time_series_data=preparar_targets(x_train,length,batch_size) 
time_series_data[0][0].shape

(64, 100, 1)

### Dados Validação

In [19]:
validation_data= preparar_targets(x_validation,length,batch_size)

validation_data[0][0].shape

(64, 100, 1)

## Rede Transformer
# https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_10_5_keras_transformers.ipynb

In [20]:
length = len(validation_data[0][0][0])
print(length)

100


In [21]:
from tensorflow import keras
from tensorflow.keras import layers

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Parte
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [22]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1)(x)
    return keras.Model(inputs, outputs)

In [23]:
input_shape = time_series_data[0][0][0].shape

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    optimizer= keras.optimizers.Adam(learning_rate=learning_rate), loss='mse', metrics='mae'
)
#model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, \
    restore_best_weights=True)]

model.fit(time_series_data, validation_data=validation_data,epochs=epocas,callbacks=callbacks)

test_targets= preparar_targets(dados_teste,length,batch_size)

model.evaluate(test_targets, verbose=1)

Epoch 1/5


KeyboardInterrupt: 